In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.functions import date_add, col, count, dayofweek, expr, hour, month

Create Spark Session

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '4g')
    .config('spark.executor.memory', '2g')
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") 
    .config('spark.executor.memoryOverhead', '3000')
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/08/19 17:48:41 WARN Utils: Your hostname, LAPTOP-RELH58H1 resolves to a loopback address: 127.0.1.1; using 172.19.22.4 instead (on interface eth0)
23/08/19 17:48:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/19 17:48:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/08/19 17:48:46 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Load Preprocessed Shooting Data

In [3]:
# Load the shooting dataset
shooting_df = spark.read.parquet("../data/raw/shooting_with_location.parquet")

Load Preprocessed Taxi Data - 3 months segments

In [4]:
# Read all Parquet files in the specified folder
sdf_yellow_2016_01_to_03 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2016_01_to_03.parquet/*')
sdf_yellow_2016_04_to_06 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2016_04_to_06.parquet/*')
sdf_yellow_2016_07_to_09 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2016_07_to_09.parquet/*')
sdf_yellow_2016_10_to_12 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2016_10_to_12.parquet/*')

sdf_yellow_2017_01_to_03 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2017_01_to_03.parquet/*')
sdf_yellow_2017_04_to_06 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2017_04_to_06.parquet/*')
sdf_yellow_2017_07_to_09 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2017_07_to_09.parquet/*')
sdf_yellow_2017_10_to_12 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2017_10_to_12.parquet/*')

sdf_yellow_2018_01_to_03 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2018_01_to_03.parquet/*')
sdf_yellow_2018_04_to_06 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2018_04_to_06.parquet/*')
sdf_yellow_2018_07_to_09 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2018_07_to_09.parquet/*')
sdf_yellow_2018_10_to_12 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2018_10_to_12.parquet/*')

sdf_yellow_2019_01_to_03 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2019_01_to_03.parquet/*')
sdf_yellow_2019_04_to_06 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2019_04_to_06.parquet/*')
sdf_yellow_2019_07_to_09 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2019_07_to_09.parquet/*')
sdf_yellow_2019_10_to_12 = spark.read.parquet('../data/raw/yellow_taxi_reduced/2019_10_to_12.parquet/*')

# List creation of all taxi data
taxi_periods = [
    sdf_yellow_2016_01_to_03,
    sdf_yellow_2016_04_to_06,
    sdf_yellow_2016_07_to_09,
    sdf_yellow_2016_10_to_12,
    sdf_yellow_2017_01_to_03,
    sdf_yellow_2017_04_to_06,
    sdf_yellow_2017_07_to_09,
    sdf_yellow_2017_10_to_12,
    sdf_yellow_2018_01_to_03,
    sdf_yellow_2018_04_to_06,
    sdf_yellow_2018_07_to_09,
    sdf_yellow_2018_10_to_12,
    sdf_yellow_2019_01_to_03,
    sdf_yellow_2019_04_to_06,
    sdf_yellow_2019_07_to_09,
    sdf_yellow_2019_10_to_12,
]

Create weekend column

In [5]:
# List to store modified DataFrames with the "weekend" column
modified_taxi_periods = []

# Loop through the list of DataFrames and add the "weekend" column
for taxi_df in taxi_periods:
    modified_taxi_df = taxi_df.withColumn("weekend", expr("(dayofweek(tpep_pickup_datetime) = 1) OR (dayofweek(tpep_pickup_datetime) = 7)").cast("int"))
    modified_taxi_periods.append(modified_taxi_df)

Create pick up hour and month column

In [6]:
# List to store modified DataFrames with the "month" and "hour" columns
timed_taxi_periods = []

# Loop through the list of DataFrames and add the "month" and "hour" columns
for taxi_df in modified_taxi_periods:
    full_modified_taxi_df = taxi_df.withColumn("Month", month(col("tpep_pickup_datetime")))
    full_modified_taxi_df = full_modified_taxi_df.withColumn("Hour", hour(col("tpep_pickup_datetime")))
    timed_taxi_periods.append(full_modified_taxi_df)


Joining Datasets

In [7]:
# List to store merged DataFrames
merged_data = []

# Loop thorugh the data frame to merge together the Dataframes
for taxi_df in timed_taxi_periods:
    merged_df = taxi_df.join(
        shooting_df,
        (taxi_df["PULocationID"] == shooting_df["LocationID"]) &
        (taxi_df["tpep_pickup_datetime"] > date_add(shooting_df["Date"], 1)) &
        (taxi_df["tpep_pickup_datetime"] <= date_add(shooting_df["Date"], 8)),
        "inner"
    )
    # Calculate the days_after_shooting column
    merged_df = merged_df.withColumn(
        "days_after_shooting",
        F.datediff(merged_df["tpep_pickup_datetime"], merged_df["Date"])
    )
    
    merged_data.append(merged_df)

Aggregating dataframe to so it focuses on number of trips per hour

Removing rows that aren't necessary

In [9]:
# List of columns to keep
columns_to_keep = ["days_after_shooting", "Hour", "Month", "tpep_pickup_datetime", "ID",
                   "Borough", "murder", "Perp_Sex", "Perp_Race", "Vict_Sex", "Vict_Race",
                   "LocationID", "Weekend","Date"]

# Create a new list of DataFrames with selected columns
select_dataframes = []
for dataframe in merged_data:
    select_dataframe = dataframe.select(*columns_to_keep)
    select_dataframes.append(select_dataframe)

Aggregating data

In [10]:
# Loop through each DataFrame in the list and perform aggregations
aggregated_dataframes = []
for dataframe in select_dataframes:
    # Select the specified columns to keep
    selected_columns = ["days_after_shooting", "Hour", "Month", "Borough", "murder", 
                        "Perp_Sex", "Perp_Race", "Vict_Sex", "Vict_Race", "LocationID", 
                        "ID", "Weekend", "Date"]
    
    # Group by pickup_hour, days_after_shooting, and ID
    grouped_data = dataframe.groupBy(*selected_columns).agg(
        count("ID").alias("number_of_trips"),
    )
    
    # Append to aggregated_dataframes list
    aggregated_dataframes.append(grouped_data)

Saving Shooting-Taxi dataframes

In [11]:
# Define a list of output paths for each Parquet file in the processed directory
output_paths = [
    "../data/curated/shooting_taxi_hour_data_reduced/2016_01_to_03.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2016_04_to_06.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2016_07_to_09.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2016_10_to_12.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2017_01_to_03.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2017_04_to_06.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2017_07_to_09.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2017_10_to_12.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2018_01_to_03.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2018_04_to_06.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2018_07_to_09.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2018_10_to_12.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2019_01_to_03.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2019_04_to_06.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2019_07_to_09.parquet",
    "../data/curated/shooting_taxi_hour_data_reduced/2019_10_to_12.parquet",
]

# Iterate through each DataFrame and its corresponding output path, then write the Parquet file
for df, output_path in zip(aggregated_dataframes, output_paths):
    df.write.mode("overwrite").parquet(output_path)

23/08/18 22:17:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/18 22:17:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/18 22:17:12 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/18 22:17:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/18 22:17:16 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/18 22:17:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/18 22:17:17 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/18 22:17:24 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/08/18 22:17:24 WARN RowBasedKeyValueBatch: Calling spill() on